# Importing the required Libraries


In [4]:
import logging

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

import azureml.core
from azureml.core import Experiment, Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
ws = Workspace.from_config()

In [5]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


quick-starts-ws-136374
aml-quickstarts-136374
southcentralus
9a7511b8-150f-4a58-8528-3e7d50216c31


# Configuring the Compute Cluster¶


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config() # this automatically looks for a directory .azureml

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4, 
                                                            idle_seconds_before_scaledown=2400,
                                                            vm_priority='lowpriority')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
ampl ComputeInstance Succeeded
cpu-cluster AmlCompute Succeeded


In [8]:
experiment_name = 'automl-Diabetes'

experiment=Experiment(ws, experiment_name)

In [9]:
experiment

Name,Workspace,Report Page,Docs Page
automl-Diabetes,quick-starts-ws-136374,Link to Azure Machine Learning studio,Link to Documentation


# Load dataset

In [19]:
# Try to load the dataset from he Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Diabetes Dataset"
description_text = "Diabetes DataSet for Capstone"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
    example_data = 'https://raw.githubusercontent.com/ohikhatemenG/Machine-Learning-Azure-Capstone/main/diabetes.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Auto Configuration

In [21]:
from azureml.train.automl import AutoMLConfig


automl_config = AutoMLConfig(
    experiment_timeout_minutes=25,
    debug_log = 'automl_errors.log',
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='Outcome',
    n_cross_validations=5,
    compute_target="cpu-cluster",
    iterations=20,
    max_concurrent_iterations=5)


In [23]:
remote_run = experiment.submit(automl_config,show_output=True)


Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_226770ad-da7c-49b8-b14c-6722b4991722

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values we

In [24]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [25]:
remote_run.wait_for_completion(show_output=True)




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_226770ad-da7c-49b8-b14c-6722b4991722',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-28T13:42:49.513568Z',
 'endTimeUtc': '2021-01-28T14:03:07.783422Z',
 'properties': {'num_iterations': '20',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-Diabetes","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-136374","workspace_name":"quick-starts-ws-136374","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":20,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"fea

In [26]:
bestrun,model=remote_run.get_output()


In [27]:
bestrun


Experiment,Id,Type,Status,Details Page,Docs Page
automl-Diabetes,AutoML_226770ad-da7c-49b8-b14c-6722b4991722_18,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.01,
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimators=25,
                           

In [29]:
import joblib
os.makedirs('./outputs', exist_ok=True)

joblib.dump(model, filename='outputs/automl_model.joblib')

model_name = bestrun.properties['model_name']
model_name


'AutoML226770add18'

In [33]:
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML model')
model.id

'AutoML226770add18:1'

In [34]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'automl-diabetes'
print(aci_service_name)


automl-diabetes


In [35]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
State: Healthy
Scoring URI: http://d41c5818-7298-48e4-9745-9a77a32620ce.southcentralus.azurecontainer.io/score


In [38]:
import requests
import json
scoring_uri='http://d41c5818-7298-48e4-9745-9a77a32620ce.southcentralus.azurecontainer.io/score'

data = {"data":
        [
          {
            "Pregnancies": 5,
            "Glucose": 155,
            "BloodPressure": 90,
            "SkinThickness": 35,
            "Insulin": 135,
            "BMI": 36.6,
            "DiabetesPedigreeFunction": 0.7,
            "Age": 55,
                      },
          {
            "Pregnancies": 3,
            "Glucose": 145,
            "BloodPressure": 95,
            "SkinThickness": 32,
            "Insulin": 155,
            "BMI": 34.3,
            "DiabetesPedigreeFunction": 0.625,
            "Age": 65,
          },
      ]
    }
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


{"result": [1, 1]}


In [47]:
df_test=df[700:710]

In [48]:
df_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
700,2,122,76,27,200,35.90,0.48,26,0
701,6,125,78,31,0,27.60,0.56,49,1
702,1,168,88,29,0,35.00,0.91,52,1
703,2,129,0,0,0,38.50,0.30,41,0
704,4,110,76,20,100,28.40,0.12,27,0


In [49]:
y_true=df_test.pop('Outcome')


In [50]:
sample_json = json.dumps({'data':df_test.to_dict(orient='records')})


In [51]:
sample_json

'{"data": [{"Pregnancies": 2, "Glucose": 122, "BloodPressure": 76, "SkinThickness": 27, "Insulin": 200, "BMI": 35.9, "DiabetesPedigreeFunction": 0.483, "Age": 26}, {"Pregnancies": 6, "Glucose": 125, "BloodPressure": 78, "SkinThickness": 31, "Insulin": 0, "BMI": 27.6, "DiabetesPedigreeFunction": 0.565, "Age": 49}, {"Pregnancies": 1, "Glucose": 168, "BloodPressure": 88, "SkinThickness": 29, "Insulin": 0, "BMI": 35.0, "DiabetesPedigreeFunction": 0.905, "Age": 52}, {"Pregnancies": 2, "Glucose": 129, "BloodPressure": 0, "SkinThickness": 0, "Insulin": 0, "BMI": 38.5, "DiabetesPedigreeFunction": 0.304, "Age": 41}, {"Pregnancies": 4, "Glucose": 110, "BloodPressure": 76, "SkinThickness": 20, "Insulin": 100, "BMI": 28.4, "DiabetesPedigreeFunction": 0.118, "Age": 27}, {"Pregnancies": 6, "Glucose": 80, "BloodPressure": 80, "SkinThickness": 36, "Insulin": 0, "BMI": 39.8, "DiabetesPedigreeFunction": 0.177, "Age": 28}, {"Pregnancies": 10, "Glucose": 115, "BloodPressure": 0, "SkinThickness": 0, "Insul

In [52]:
output = service.run(sample_json)

In [53]:
output

'{"result": [0, 0, 1, 1, 0, 0, 0, 0, 1, 0]}'

In [54]:
print("Predicted Values are :", output)
print("True Values are:",y_true.values)

Predicted Values are : {"result": [0, 0, 1, 1, 0, 0, 0, 0, 1, 0]}
True Values are: [0 1 1 0 0 0 1 0 1 1]


In [55]:
service.get_logs()

'2021-01-28T15:19:53,525909126+00:00 - nginx/run \n2021-01-28T15:19:53,527563467+00:00 - iot-server/run \n2021-01-28T15:19:53,527582268+00:00 - rsyslog/run \n2021-01-28T15:19:53,535230817+00:00 - gunicorn/run \n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [56]:
service.delete()
